# Exploring 15 records from 1543 (Kickback from Lex's R Code)
***
***DAEN690***

***George Mason University***

***Author:*** Grace Cox (Team LEGO)

***Date:*** November 15, 2021
***
`This Jupyter Notebook explores 15 records (of the 1543 OTHER records) that were returned after being ran through Lex's R Code. The process for exploring these records is as follows:`

1. Explore the 'DATASET' field, which indicates what type of Origin Location Reference appears within the 'REMARKS' field for each record (e.g., AIRPORT, NAVAID, AIRPORT DIRECTLY OVER, NAVAID DIRECTLY OVER, RUNWAY, FIX RADIAL, HELIPORT, etc) 
<br>

2. For each of the unique Origin Location References that appear in the 'DATASET' field:

    a. Run the uas_loc_parts() function of the dataframe 
    
    b. Link to the appropriate Origin Location Reference linking function (e.g., link_airportsCleaned_ident(), link_airportsCleaned_icao(), nav_link())
        i. Separate the field that were successfully linked with the Origin Location Reference Dataset(s) and those that were not. Those that were not linked successfully will be moved to the exceptions file for further review.
    c. Run the successfully linked dataframe through the uas_lat_long() function to calculate the UAS Latitude/Longitude coordinates for each respective record/remark. 

***
### INPUT: 
*Dataframe (read from a .csv file) containing 15 records, of 1543 records that did not hit on the initial 'XX NM dir XXX' regular expressions, and was not initially picked up by our Runways regular expressions.*

### OUTPUT:
<font color='red'>**EXCPETIONS FILE**</font>

*City: 7 records*

<font color='green'>**UAS LATITUDE/LONGITUDE COORDINATE CALCULATED**</font>

*FAA Airport Identifier: 6 records*

*CEDAR Lat/Long: 2 records*


## Import Statements

In [2]:
# Import Statements
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import chart_studio.plotly as py
import re
import geopy
from geopy.distance import geodesic

from IPython.display import display, HTML

In [3]:
output13 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Grace - All_Points_Standard_13OUTOF543.csv')
output13

,index,REMARKS,UAS_Location_Runways,dateonly,timeonly,DATE,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET,cityname
0,2022,Aircraft reported a red and white quad copter ...,UNKN,5/19/2019,5:04,2019-05-19T05:04Z,DEN,UAS,NaN,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 NM S,CITY,Denton
1,166,Aircraft observed a 4 rotor uas off of the 12 ...,UNKN,7/30/2019,17:47,2019-07-30T17:47Z,DEN,UAS,NaN,NCT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM W,CITY,Reno
2,188,Aircraft observed a uas off the left side at s...,UNKN,8/5/2019,15:12,2019-08-05T15:12Z,DEN,UAS,NaN,PHX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25 NM N,CITY,Phoenix
3,326,Aircraft observed a red and black uas while E ...,UNKN,11/3/2019,17:24,2019-11-03T17:24Z,DEN,UAS,NaN,IAH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM W,CITY,Houston
4,387,Aircraft observed a uas 300 feet below while E...,UNKN,1/21/2020,18:00,2020-01-21T18:00Z,DEN,UAS,NaN,TOA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 NM S,CITY,Torrance
5,603,Aircraft observed a silver uas while E bound a...,UNKN,6/22/2020,16:20,2020-06-22T16:20Z,NCRCC,UAS,NaN,HEF,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4 NM W of HEF,AIRPORT,NaN
6,652,Aircraft observed a uas off the right side whi...,UNKN,7/12/2020,17:51,2020-07-12T17:51Z,DEN,UAS,NaN,AZO,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6 NM S of AZO,AIRPORT,NaN
7,719,Aircraft observed a uas at the same altitude w...,UNKN,8/16/2020,16:45,2020-08-16T16:45Z,DEN,UAS,NaN,RFD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8 NM N,CITY,Rockford
8,723,Aircraft observed a uas at same altitude W bou...,UNKN,8/16/2020,20:44,2020-08-16T20:44Z,DEN,UAS,NaN,R90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM S,CITY,Omaha
9,982,Aircraft observed a uas off the front side whi...,UNKN,2/17/2021,15:01,2021-02-17T15:01Z,NCRCC,UAS,NaN,PCT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 NM W of RIC,AIRPORT,NaN


In [5]:
output13_df = output13[['REMARKS', 'DATASET', 'UASLOCATION', 'cityname']]
output13_df

,REMARKS,DATASET,UASLOCATION,cityname
0,Aircraft reported a red and white quad copter ...,CITY,3 NM S,Denton
1,Aircraft observed a 4 rotor uas off of the 12 ...,CITY,5 NM W,Reno
2,Aircraft observed a uas off the left side at s...,CITY,25 NM N,Phoenix
3,Aircraft observed a red and black uas while E ...,CITY,5 NM W,Houston
4,Aircraft observed a uas 300 feet below while E...,CITY,1 NM S,Torrance
5,Aircraft observed a silver uas while E bound a...,AIRPORT,4 NM W of HEF,NaN
6,Aircraft observed a uas off the right side whi...,AIRPORT,6 NM S of AZO,NaN
7,Aircraft observed a uas at the same altitude w...,CITY,8 NM N,Rockford
8,Aircraft observed a uas at same altitude W bou...,CITY,5 NM S,Omaha
9,Aircraft observed a uas off the front side whi...,AIRPORT,3 NM W of RIC,NaN


In [6]:
# Determine Unique Origin Location References within the 15 record kickback
output13_df['DATASET'].unique()

array(['CITY', 'AIRPORT'], dtype=object)

# AIRPORTS

`Origin Location for UAS is recognized as an Airport`

In [7]:
output13_airport = output13_df[output13_df['DATASET'] == 'AIRPORT'].reset_index()
output13_airport

,index,REMARKS,DATASET,UASLOCATION,cityname
0,5,Aircraft observed a silver uas while E bound a...,AIRPORT,4 NM W of HEF,NaN
1,6,Aircraft observed a uas off the right side whi...,AIRPORT,6 NM S of AZO,NaN
2,9,Aircraft observed a uas off the front side whi...,AIRPORT,3 NM W of RIC,NaN
3,10,Aircraft observed a grey uas off the right sid...,AIRPORT,5 NM E of LAX,NaN
4,11,"Aircraft observed a uas off the right side, 50...",AIRPORT,10 NM E of ORD,NaN
5,12,Aircraft observed a uas off the left side whil...,AIRPORT,8.5 NM E of PHL,NaN


In [8]:
def uas_loc_parts(dataframe):
    '''
    This function takes the 'UASLOCATION' column from Lex's 'Kickbacks' and breaks it into the different parts of the 
    UAS Location, to include: Distance (NM), Direction Abbreviation, Airport Identifier
    
    input : dataframe is a df that includes the 'UASLOCATION' field
    output : the dataframe provided as input to the user with additional fields to include: 'Distance_NM', 'Bearing', 
                                                                                            'IDENT'
    '''
    uasLoc_col = dataframe['UASLOCATION']
    dist_list = []
    bearing_list = []
    ident_list = []
    dist_kilo = []
    bearDegree = []
    
    # Dictionary of Bearing Abbreviation to Degrees Conversions
    bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
    }
    
    beardeg_list = list(bearing_deg.keys())
    
    for i in range(len(dataframe)):
        # Get Distance (NM) from UAS LOCATION and Convert Distance to Kilometers
        distNM = re.sub("[^\d\.]","", str(re.findall(r"[-+]?\d*\.\d+|\d+", uasLoc_col[i].split()[0])))
        if distNM == '':
            dist_list.append(0)
            dist_kilo.append(0.00)
        else:
            dist_list.append(distNM)
            dist_kilo.append(float(distNM) * 1.852)
        
        # Get IDENT from UASLOCATION
        ident = uasLoc_col[i].split()[-1].replace('of', '')
        ident_list.append(ident)
        
        # Get Bearing from UASLOCATION and Convert Bearing to Degrees
        #bearing = uasLoc_col[i].replace('NM', '').replace('of','').split()[-2]
        bearing = re.sub('(NM|of|OF)', ' ', uasLoc_col[i]).split()[-2]
        bearing_list.append(bearing)
        if bearing.isdigit():
            bearDegree.append(bearing)
        else:
            bearDegree.append(bearing_deg.get(bearing))
        
    # Append new fields to dataframe provided as input    
    dataframe['Distance_NM'] = dist_list
    dataframe['Distance_Kilometers'] = dist_kilo
    dataframe['Bearing'] = bearing_list
    dataframe['Bearing_Degrees'] = bearDegree
    dataframe['IDENT'] = ident_list
    
    
    return dataframe

In [10]:
air13 = uas_loc_parts(output13_airport)
air13

,index,REMARKS,DATASET,UASLOCATION,cityname,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT
0,5,Aircraft observed a silver uas while E bound a...,AIRPORT,4 NM W of HEF,NaN,4,7.408,W,270,HEF
1,6,Aircraft observed a uas off the right side whi...,AIRPORT,6 NM S of AZO,NaN,6,11.112,S,180,AZO
2,9,Aircraft observed a uas off the front side whi...,AIRPORT,3 NM W of RIC,NaN,3,5.556,W,270,RIC
3,10,Aircraft observed a grey uas off the right sid...,AIRPORT,5 NM E of LAX,NaN,5,9.260,E,90,LAX
4,11,"Aircraft observed a uas off the right side, 50...",AIRPORT,10 NM E of ORD,NaN,10,18.520,E,90,ORD
5,12,Aircraft observed a uas off the left side whil...,AIRPORT,8.5 NM E of PHL,NaN,8.5,15.742,E,90,PHL


In [13]:
def link_airportsCleaned(dataframe):
    '''
    This function links the provided dataframe to the airports_cleaned.csv file.
    
    input : dataframe is a df with an 'IDENT' field on which we will link to the airports_cleaned.csv
    output : the dataframe provided as input with airport lat/long fields appended
    '''
    
    # Read in Cleaned Airports dataset
    airportsC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/airports_cleaned.csv')
    
    return pd.merge(dataframe, airportsC, on='IDENT', how='left')

In [14]:
air13_link = link_airportsCleaned(air13)
air13_link

,index,REMARKS,DATASET,UASLOCATION,cityname,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID
0,5,Aircraft observed a silver uas while E bound a...,AIRPORT,4 NM W of HEF,NaN,4,7.408,W,270,HEF,9393671A-A959-40DA-B859-35EB3D1C972F,Manassas Rgnl/Harry P Davis Fld,38.721023,-77.515101,KHEF
1,6,Aircraft observed a uas off the right side whi...,AIRPORT,6 NM S of AZO,NaN,6,11.112,S,180,AZO,FAB7EE32-A0A3-49FD-8070-CB13B77D1D29,Kalamazoo/Battle Creek Intl,42.234389,-85.551556,KAZO
2,9,Aircraft observed a uas off the front side whi...,AIRPORT,3 NM W of RIC,NaN,3,5.556,W,270,RIC,69E61D8F-915F-4B1D-9666-6338F1F19CD9,Richmond Intl,37.505181,-77.319739,KRIC
3,10,Aircraft observed a grey uas off the right sid...,AIRPORT,5 NM E of LAX,NaN,5,9.260,E,90,LAX,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,Los Angeles Intl,33.942496,-118.408049,KLAX
4,11,"Aircraft observed a uas off the right side, 50...",AIRPORT,10 NM E of ORD,NaN,10,18.520,E,90,ORD,8719978B-8B62-4688-A0AE-46450A687C68,Chicago O'hare Intl,41.976629,-87.907253,KORD
5,12,Aircraft observed a uas off the left side whil...,AIRPORT,8.5 NM E of PHL,NaN,8.5,15.742,E,90,PHL,43D58CEC-68DC-44C1-A241-0A33C0F29415,Philadelphia Intl,39.872084,-75.240663,KPHL


In [15]:
def uas_lat_long(dataframe):  
    '''
    This function takes in a .csv file (string) of SkyWatch records for which the Remarks are in the Standard Format
    and uses the bearing_dist_originIdent() function to output the dataframe containing complete UAS Location information
    in terms of the UAS' Latitude and Longitude.
    
    @param dataframe : The dataframe that includes the fields 'LATITUDE', 'LONGITUDE', 'Bearing_Degrees', 
                        'Distance_Kilometers' to have UAS Lat/Long values calculated for
    
    @output new_df: returns a dataframe containing the Remark, UAS Location, FAA Airport Identifier, Distance (in NM) from 
                    the airport, and bearing abbreviation (WHERE ALL BEARINGS ARE VALID)
                
                *** it should be noted that the full dataset that includes remarks for which UAS latitudes and longitudes
                    were calculated is located in the StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv file.
                    
    
    '''
    uas_lat = []
    uas_long = []
    
    for i in range(len(dataframe)):
        lat_airport = pd.to_numeric(dataframe['LATITUDE'][i])
        long_airport = pd.to_numeric(dataframe['LONGITUDE'][i])
        b = pd.to_numeric(dataframe['Bearing_Degrees'][i])
        d = pd.to_numeric(dataframe['Distance_Kilometers'][i])

        origin = geopy.Point(lat_airport, long_airport)
        destination = geodesic(kilometers=d).destination(origin,b)

        lat2, lon2 = destination.latitude, destination.longitude

        uas_lat.append(lat2)
        uas_long.append(lon2)
        
    
    # Append UAS Lat/Long information to DataFrame
    dataframe['UAS_Latitude'] = uas_lat
    dataframe['UAS_Longitude'] = uas_long
    
    # Export final dataframe (EXCLUDING NULL BEARING INFO) to .csv file
    #dataframe.to_csv('StandardRemark_UAS_LatLong_validBearing_FAAairIdent.csv', index = False)
    
    return dataframe

In [16]:
air13_uas = uas_lat_long(air13_link)
air13_uas

,index,REMARKS,DATASET,UASLOCATION,cityname,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,IDENT,GLOBAL_ID,NAME,LATITUDE,LONGITUDE,ICAO_ID,UAS_Latitude,UAS_Longitude
0,5,Aircraft observed a silver uas while E bound a...,AIRPORT,4 NM W of HEF,NaN,4,7.408,W,270,HEF,9393671A-A959-40DA-B859-35EB3D1C972F,Manassas Rgnl/Harry P Davis Fld,38.721023,-77.515101,KHEF,38.720992,-77.600284
1,6,Aircraft observed a uas off the right side whi...,AIRPORT,6 NM S of AZO,NaN,6,11.112,S,180,AZO,FAB7EE32-A0A3-49FD-8070-CB13B77D1D29,Kalamazoo/Battle Creek Intl,42.234389,-85.551556,KAZO,42.134350,-85.551556
2,9,Aircraft observed a uas off the front side whi...,AIRPORT,3 NM W of RIC,NaN,3,5.556,W,270,RIC,69E61D8F-915F-4B1D-9666-6338F1F19CD9,Richmond Intl,37.505181,-77.319739,KRIC,37.505164,-77.382576
3,10,Aircraft observed a grey uas off the right sid...,AIRPORT,5 NM E of LAX,NaN,5,9.260,E,90,LAX,2C5866F1-5E27-4032-9AF8-988CE5DB82BA,Los Angeles Intl,33.942496,-118.408049,KLAX,33.942456,-118.307883
4,11,"Aircraft observed a uas off the right side, 50...",AIRPORT,10 NM E of ORD,NaN,10,18.520,E,90,ORD,8719978B-8B62-4688-A0AE-46450A687C68,Chicago O'hare Intl,41.976629,-87.907253,KORD,41.976411,-87.683801
5,12,Aircraft observed a uas off the left side whil...,AIRPORT,8.5 NM E of PHL,NaN,8.5,15.742,E,90,PHL,43D58CEC-68DC-44C1-A241-0A33C0F29415,Philadelphia Intl,39.872084,-75.240663,KPHL,39.871938,-75.056660


In [17]:
# Export to .csv 
air13_uas.to_csv('airport6OF13OF543_uasLatLong.csv', index = False)

# CITY NAME

`Origin Location Reference is a City Name, not an airport.
We want to link these city names with the respective NAVAID Systems.`

In [7]:
output13_city = output13[output13['DATASET'] == 'CITY'].reset_index(drop=True)
output13_city

,index,REMARKS,UAS_Location_Runways,dateonly,timeonly,DATE,POD,PRIMARYCODE,SECONDARYCODES,REPORTINGFACILITY,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,RWYLOCATION,UASLOCATION,DATASET,cityname
0,2022,Aircraft reported a red and white quad copter ...,UNKN,5/19/2019,5:04,2019-05-19T05:04Z,DEN,UAS,NaN,DFW,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3 NM S,CITY,Denton
1,166,Aircraft observed a 4 rotor uas off of the 12 ...,UNKN,7/30/2019,17:47,2019-07-30T17:47Z,DEN,UAS,NaN,NCT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM W,CITY,Reno
2,188,Aircraft observed a uas off the left side at s...,UNKN,8/5/2019,15:12,2019-08-05T15:12Z,DEN,UAS,NaN,PHX,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25 NM N,CITY,Phoenix
3,326,Aircraft observed a red and black uas while E ...,UNKN,11/3/2019,17:24,2019-11-03T17:24Z,DEN,UAS,NaN,IAH,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM W,CITY,Houston
4,387,Aircraft observed a uas 300 feet below while E...,UNKN,1/21/2020,18:00,2020-01-21T18:00Z,DEN,UAS,NaN,TOA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1 NM S,CITY,Torrance
5,719,Aircraft observed a uas at the same altitude w...,UNKN,8/16/2020,16:45,2020-08-16T16:45Z,DEN,UAS,NaN,RFD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8 NM N,CITY,Rockford
6,723,Aircraft observed a uas at same altitude W bou...,UNKN,8/16/2020,20:44,2020-08-16T20:44Z,DEN,UAS,NaN,R90,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5 NM S,CITY,Omaha


In [9]:
# Export to .csv 
output13_city.to_csv('city07OF13_EXCEPTIONS.csv', index = False)

In [28]:
output13_city = output13_df[output13_df['DATASET'] == 'CITY'].drop(columns = ['Bearing', 'Bearing_Degrees', 'IDENT', 'Distance_NM', 'Distance_Kilometers']).reset_index(drop=True)
output13_city

,REMARKS,DATASET,UASLOCATION,cityname
0,Aircraft reported a red and white quad copter ...,CITY,3 NM S,Denton
1,Aircraft observed a 4 rotor uas off of the 12 ...,CITY,5 NM W,Reno
2,Aircraft observed a uas off the left side at s...,CITY,25 NM N,Phoenix
3,Aircraft observed a red and black uas while E ...,CITY,5 NM W,Houston
4,Aircraft observed a uas 300 feet below while E...,CITY,1 NM S,Torrance
5,Aircraft observed a uas at the same altitude w...,CITY,8 NM N,Rockford
6,Aircraft observed a uas at same altitude W bou...,CITY,5 NM S,Omaha


In [53]:
def uas_loc_partsCITY(dataframe):
    '''
    This function takes the 'UASLOCATION' column from Lex's 'Kickbacks' and breaks it into the different parts of the 
    UAS Location, to include: Distance (NM), Direction Abbreviation, Airport Identifier
    
    input : dataframe is a df that includes the 'UASLOCATION' field
    output : the dataframe provided as input to the user with additional fields to include: 'Distance_NM', 'Bearing', 
                                                                                            'IDENT'
    '''
    uasLoc_col = dataframe['UASLOCATION']
    dist_list = []
    bearing_list = []
    city_list = []
    dist_kilo = []
    bearDegree = []
    
    # Dictionary of Bearing Abbreviation to Degrees Conversions
    bearing_deg = {
    'N' : 0,
    'NNE' : 23,
    'NE' : 45,
    'ENE' : 68,
    'E' : 90,
    'ESE' : 113,
    'SE' : 135,
    'SSE' : 158,
    'S' : 180,
    'SSW' : 203,
    'SW' : 225,
    'WSW' : 248,
    'W' : 270,
    'WNW' : 293,
    'NW' : 315,
    'NNW' : 338
    }
    
    beardeg_list = list(bearing_deg.keys())
    
    for i in range(len(dataframe)):
        # Get Distance (NM) from UAS LOCATION and Convert Distance to Kilometers
        distNM = re.sub("[^\d\.]","", str(re.findall(r"[-+]?\d*\.\d+|\d+", uasLoc_col[i].split()[0])))
        if distNM == '':
            dist_list.append(0)
            dist_kilo.append(0.00)
        else:
            dist_list.append(distNM)
            dist_kilo.append(float(distNM) * 1.852)
        
        # Get CITY uppercase
        dataframe['cityname'].loc[i] = dataframe['cityname'][i].upper()
        
        # Get Bearing from UASLOCATION and Convert Bearing to Degrees
        #bearing = uasLoc_col[i].replace('NM', '').replace('of','').split()[-2]
        
        bearing = uasLoc_col[i].split()[-1]
        bearing_list.append(bearing)
        if bearing.isdigit():
            bearDegree.append(bearing)
        else:
            bearDegree.append(bearing_deg.get(bearing))
        
    # Append new fields to dataframe provided as input    
    dataframe['Distance_NM'] = dist_list
    dataframe['Distance_Kilometers'] = dist_kilo
    dataframe['Bearing'] = bearing_list
    dataframe['Bearing_Degrees'] = bearDegree
    dataframe['NAME_TXT'] = dataframe['cityname']
    
    
    return dataframe

In [54]:
uas_loc_partsCITY(output13_city)

C:\Users\grace\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,REMARKS,DATASET,UASLOCATION,cityname,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,NAME_TXT,CITY
0,Aircraft reported a red and white quad copter ...,CITY,3 NM S,DENTON,3,5.556,S,180,DENTON,DENTON
1,Aircraft observed a 4 rotor uas off of the 12 ...,CITY,5 NM W,RENO,5,9.260,W,270,RENO,RENO
2,Aircraft observed a uas off the left side at s...,CITY,25 NM N,PHOENIX,25,46.300,N,0,PHOENIX,PHOENIX
3,Aircraft observed a red and black uas while E ...,CITY,5 NM W,HOUSTON,5,9.260,W,270,HOUSTON,HOUSTON
4,Aircraft observed a uas 300 feet below while E...,CITY,1 NM S,TORRANCE,1,1.852,S,180,TORRANCE,TORRANCE
5,Aircraft observed a uas at the same altitude w...,CITY,8 NM N,ROCKFORD,8,14.816,N,0,ROCKFORD,ROCKFORD
6,Aircraft observed a uas at same altitude W bou...,CITY,5 NM S,OMAHA,5,9.260,S,180,OMAHA,OMAHA


In [55]:
def link_navaidCleaned_CITY(dataframe):
    '''
    This function links the provided dataframe to the NAVAID_cleaned.csv file.
    
    input : dataframe is a df with an 'IDENT' field on which we will link to the airports_cleaned.csv
    output : the dataframe provided as input with navaid lat/long fields appended
    '''
    
    # Read in Cleaned Airports dataset
    navC = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/NAVAID_cleaned.csv')
    
    return pd.merge(dataframe, navC, on='NAME_TXT', how='left')

In [56]:
link_navaidCleaned_CITY(uas_loc_partsCITY(output13_city))

C:\Users\grace\anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


,REMARKS,DATASET,UASLOCATION,cityname,Distance_NM,Distance_Kilometers,Bearing,Bearing_Degrees,NAME_TXT,CITY_x,LONGITUDE,LATITUDE,IDENT,CITY_y
0,Aircraft reported a red and white quad copter ...,CITY,3 NM S,DENTON,3,5.556,S,180,DENTON,DENTON,NaN,NaN,NaN,NaN
1,Aircraft observed a 4 rotor uas off of the 12 ...,CITY,5 NM W,RENO,5,9.260,W,270,RENO,RENO,NaN,NaN,NaN,NaN
2,Aircraft observed a uas off the left side at s...,CITY,25 NM N,PHOENIX,25,46.300,N,0,PHOENIX,PHOENIX,-111.970210,33.433026,PXR,PHOENIX
3,Aircraft observed a red and black uas while E ...,CITY,5 NM W,HOUSTON,5,9.260,W,270,HOUSTON,HOUSTON,-126.650898,54.452350,YYD,NaN
4,Aircraft observed a uas 300 feet below while E...,CITY,1 NM S,TORRANCE,1,1.852,S,180,TORRANCE,TORRANCE,NaN,NaN,NaN,NaN
5,Aircraft observed a uas at the same altitude w...,CITY,8 NM N,ROCKFORD,8,14.816,N,0,ROCKFORD,ROCKFORD,-89.199278,42.225586,RFD,CHICAGO/ROCKFORD
6,Aircraft observed a uas at same altitude W bou...,CITY,5 NM S,OMAHA,5,9.260,S,180,OMAHA,OMAHA,-95.736861,41.167174,OVR,MINEOLA


# CEDAR LAT/LONG

`The UAS Location is provided in the CEDAR Remarks and is given as lat/long coordinates`

In [59]:
output2 = pd.read_csv('C:/Users/grace/OneDrive/Desktop/GMU/DAEN690/Grace - All_Points_Standard_Runways_2OUTOF263.csv')
output2['UAS_Longitude'] = output2['UASLONG.CEDAR']
output2['UAS_Latitude'] = output2['UASLAT.CEDAR']
output2

,index,REMARKS,UAS_Location_Runways,UASLOCATION,IDENT,RWYLOCATION,UAS_Distance,dateonly,timeonly,DATE,...,UASDIM.CEDAR,UASFWROTOR.CEDAR,PILOTREPORTEDNMAC.CEDAR,TCASRA.CEDAR,SUMMARY.CEDAR,QAFINDINGS.CEDAR,DATASET,cityname,UAS_Longitude,UAS_Latitude
0,413,Aircraft observed a uas on a 2-3 NM final to R...,['RWY25L'],NaN,NaN,RWY 25L,['3 NM'],7/18/2021,20:01,2021-07-18T20:01Z,...,UAS Dimensions (feet): NA,NaN,No,No,"UNAUTHORIZED UAS ACTIVITY RY25L: NKS1157, A319...",Reviewed MOR Summary and Falcon for a UAS repo...,LAT/LONG,NaN,-118.3400,33.9400
1,425,Aircraft observed a white uas 500 feet below t...,['RWY22R'],NaN,NaN,RWY 22R,['10NM'],8/6/2021,23:19,2021-08-06T23:19Z,...,UAS Dimensions (feet): 1,Rotor Craft,No,No,FDX505 REPORTED A SMALL WHITE QUADROCOPTER 500...,QA reviewed. Not reported as NMAC.,LAT/LONG,NaN,-83.2952,42.3656


In [60]:
# Export to .csv file 
output2.to_csv('2LLof263_uasLatLong.csv', index=False)